<a href="https://colab.research.google.com/github/SeanBarnier/HAFS_Air-Sea/blob/main/getHAFSASlices.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This program subsets and saves portions of output from the MOM6 model along the track of a given TC.

#Set up environment

In [ ]:
!pip install cfgrib

In [ ]:
!sudo apt install aria2

In [ ]:
import xarray as xr
import pandas as pd
from datetime import datetime as dt
import cfgrib
import subprocess as sp

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

#User parameters

In [ ]:
subfolder = "RI"
saveLocAtm = "drive/MyDrive/savedData/hafsaOutput/" + subfolder
saveLocOce = "drive/MyDrive/savedData/mom6Output/" + subfolder
bucket = "https://noaa-nws-hafs-pds.s3.amazonaws.com/hfsa/"

name = "Milton"
tcNum = "14"
trackType = ""

start = dt(year=2024, month=10, day=8, hour=6)
end = dt(year=2024, month=10, day=9, hour=0)

fHourStep = 3 #Normally 3 for HAFS-A
forecastLength = 126 #Normally 126 for HAFS-A. Changeable for testing.
runStep = 6 #Normally 6 for HAFS-A

downloadAtm = False
downloadOce = True
replaceExisting = False

In [ ]:
atmTop = 700 # in hPa
oceBottom = 530 # In m below surface. This was chosen to include a layer in the files that's around 529 m

#Determine Area and Temporal Extent

Get best track data and find bounds of TC track

In [ ]:
bt = pd.read_csv(f"/content/drive/MyDrive/savedData/{name}/hurdat2_" + name + trackType + ".csv")
latBounds = [min(bt.lat), max(bt.lat)]
lonBounds = [min(bt.lon), max(bt.lon)]

In [ ]:
dateFormat = "%Y-%m-%d %H:%M:%S"
runFormat = "%Y%m%d%H"

Find times needed

In [ ]:
fcastTimes = {} #Key: initiation, item: valid time list

initTime = start
while initTime <= end:
  validTime = initTime
  fcastTimes[initTime] = []
  fhour = 0

  while fhour <= forecastLength:
    fcastTimes[initTime].append(validTime)
    validTime += pd.Timedelta(hours=fHourStep)
    fhour += fHourStep

  initTime += pd.Timedelta(hours=runStep)

In [ ]:
t = [fcastTimes[key] for key in fcastTimes.keys()]
flat_list = [item for sublist in t for item in sublist]
totalAtmData = 200 * len(flat_list)
totalOceData = 30 * len(flat_list)
print(f"Approximate storage needed: {(totalAtmData+totalOceData)/1000} GB")

In [ ]:
fcastTimes

#Retrieve Data

Determine what files already exist

In [ ]:
existingFilesLs = !ls {saveLocAtm}
existingFiles = [f for files in existingFilesLs for f in files.split("  ")]
existingFileNames = [f.replace("nc", "grb2").replace("hafsa", "atm").replace("mom6", "oce") for f in existingFiles]
existingFileNames.sort()

Download HAFS

In [ ]:
for init, validList in fcastTimes.items():

  if not downloadAtm: break

  initDate, initHour = init.strftime("%Y%m%d_%H").split("_")

  for valid in validList:

    fhour = str(int((valid-init).total_seconds() / 3600))
    while len(fhour) < 3: fhour = "0" + fhour

    atmURL = bucket + initDate + "/" + initHour + "/" + tcNum + "l." + initDate + initHour + ".hfsa.storm.atm.f" + fhour + ".grb2"
    atmFile = "atm_" + initDate + initHour + "_f" + fhour + ".grb2"
    if atmFile in existingFileNames and not replaceExisting:
      print(atmFile + " already exists.\n")
      continue #Skip files we already have

    !aria2c -x 16 -s 16 --allow-overwrite=true -o {atmFile} {atmURL}
    atmData = xr.open_dataset(atmFile, engine="cfgrib", decode_timedelta=True, filter_by_keys={'stepType': 'instant', 'typeOfLevel': 'isobaricInhPa'})

    pressureSlice = slice(max(atmData.isobaricInhPa.data), atmTop)
    #Longitude in atm files are in degrees east, but are -180 - 180 in oce files. point has them from -180 - 180
    atmSlice = atmData.sel(latitude=slice(latBounds[0], latBounds[1]), longitude=slice(360+lonBounds[0], 360+lonBounds[1]), isobaricInhPa=pressureSlice)

    atmSlice.to_netcdf(saveLocAtm + "/hafsa_" + initDate + initHour + "_f" + fhour + ".nc")

    !rm {atmFile}

Download MOM6

In [ ]:
existingFilesLs = !ls {saveLocOce}
existingFiles = [f for files in existingFilesLs for f in files.split("  ")]
existingFileNames = [f.replace("nc", "grb2").replace("hafsa", "atm").replace("mom6", "oce") for f in existingFiles]
existingFileNames.sort()

In [ ]:
missingFiles = ['oce_2024100800_f000.nc', "oce_2024100800_f126.nc"]

In [ ]:
for init, validList in fcastTimes.items():

  if not downloadOce: break

  initDate, initHour = init.strftime("%Y%m%d_%H").split("_")
  oceRun = False

  for valid in validList:

    fhour = str(int((valid-init).total_seconds() / 3600))
    while len(fhour) < 3: fhour = "0" + fhour

    oceURL = bucket + initDate + "/" + initHour + "/" + tcNum + "l." + initDate + initHour + ".hfsa.mom6.f" + fhour + ".nc"
    oceFile = "oce_" + initDate + initHour + "_f" + fhour + ".nc"

    if atmFile in existingFileNames and not replaceExisting:
      print(atmFile + " already exists.\n")
      continue #Skip files we already have

    if oceFile in missingFiles: continue #This file is missing
    !aria2c -x 16 -s 16 --allow-overwrite=true -o {oceFile} {oceURL}
    oceData = xr.open_dataset(oceFile, decode_times=False)

    depthSlice = slice(min(oceData.z_l.data), oceBottom)
    oceSlice = oceData.sel(z_l=depthSlice, z_i=depthSlice,\
                           xh=slice(lonBounds[0],lonBounds[1]), yh=slice(latBounds[0],latBounds[1]),\
                           xq=slice(lonBounds[0],lonBounds[1]), yq=slice(latBounds[0],latBounds[1]))

    oceSlice.to_netcdf(saveLocOce + "/mom6_" + initDate + initHour + "_f" + fhour + ".nc")

    !rm {oceFile}